In [ ]:
import pandas as pd
import numpy as np
import gzip
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import matplotlib.colors as colors
%matplotlib inline
from datetime import datetime
import dask.bag as db

In [ ]:
product_reviews_hp4 = pd.read_csv('datasets\Product_reviews_headphones.csv')

In [ ]:
product_reviews_hp4['review_text'] = product_reviews_hp4[['summary', 'reviewText']].apply(lambda x: " ".join(str(y) for y in x if str(y) != 'nan'), axis = 1)
product_reviews_hp4 = product_reviews_hp4.drop(['reviewText', 'summary'], axis = 1)
product_reviews_hp4.head()

,Rating,verified,reviewTime,reviewerID,asin,reviewerName,unixReviewTime,category,tech1,description,...,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,review_text
0,5,True,"02 22, 2015",A38RQFVQ1AKJQQ,4126895493,George Walker,1424563200,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,...,NaN,HeadGear,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Five Stars Great headphones. It's just the cor...
1,5,True,"05 8, 2017",A299MRB9O6GWDE,4126895493,Carolyn B,1494201600,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,...,NaN,HeadGear,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Officewear Really like these headphone. Wante...
2,1,True,"11 5, 2016",A3ACFC6DQQLIQT,4126895493,MK,1478304000,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,...,NaN,HeadGear,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,For the money they are fine. Just hope they ho...
3,3,True,"09 24, 2016",A36BC0YFDBNB5X,4126895493,bigboy,1474675200,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,...,NaN,HeadGear,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Three Stars Very good
4,1,True,"07 17, 2016",A212PQ0HQPNNWM,4126895493,Kelly Hales,1468713600,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,...,NaN,HeadGear,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Currently returning this product because the s...


In [ ]:
product_reviews_hp4['time'] = product_reviews_hp4.reviewTime.str.replace(',', "")
product_reviews_hp4['time'] = pd.to_datetime(product_reviews_hp4['time'], format = '%m %d %Y')

# Drop redundant 'reviewTime' column
product_reviews_hp4 = product_reviews_hp4.drop('reviewTime', axis = 1)
product_reviews_hp4.head()

,Rating,verified,reviewerID,asin,reviewerName,unixReviewTime,category,tech1,description,fit,...,brand,feature,rank,also_view,main_cat,similar_item,date,price,review_text,time
0,5,True,A38RQFVQ1AKJQQ,4126895493,George Walker,1424563200,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,HeadGear,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Five Stars Great headphones. It's just the cor...,2015-02-22
1,5,True,A299MRB9O6GWDE,4126895493,Carolyn B,1494201600,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,HeadGear,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Officewear Really like these headphone. Wante...,2017-05-08
2,1,True,A3ACFC6DQQLIQT,4126895493,MK,1478304000,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,HeadGear,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,For the money they are fine. Just hope they ho...,2016-11-05
3,3,True,A36BC0YFDBNB5X,4126895493,bigboy,1474675200,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,HeadGear,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Three Stars Very good,2016-09-24
4,1,True,A212PQ0HQPNNWM,4126895493,Kelly Hales,1468713600,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,HeadGear,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Currently returning this product because the s...,2016-07-17


In [ ]:
good_rate = len(product_reviews_hp4[product_reviews_hp4['Rating'] >= 3])
bad_rate = len(product_reviews_hp4[product_reviews_hp4['Rating'] < 3])

# Printing rates and their total numbers
print ('Good ratings : {} reviews for Heaphones products'.format(good_rate))
print ('Bad ratings : {} reviews for Headphones products'.format(bad_rate))

Good ratings : 43964 reviews for Heaphones products
Bad ratings : 5679 reviews for Headphones products


In [ ]:
product_reviews_hp4['rating_class'] = product_reviews_hp4['Rating'].apply(lambda x: 'bad' if x < 3 else'good')
product_reviews_hp4.head()

,Rating,verified,reviewerID,asin,reviewerName,unixReviewTime,category,tech1,description,fit,...,feature,rank,also_view,main_cat,similar_item,date,price,review_text,time,rating_class
0,5,True,A38RQFVQ1AKJQQ,4126895493,George Walker,1424563200,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Five Stars Great headphones. It's just the cor...,2015-02-22,good
1,5,True,A299MRB9O6GWDE,4126895493,Carolyn B,1494201600,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Officewear Really like these headphone. Wante...,2017-05-08,good
2,1,True,A3ACFC6DQQLIQT,4126895493,MK,1478304000,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,For the money they are fine. Just hope they ho...,2016-11-05,bad
3,3,True,A36BC0YFDBNB5X,4126895493,bigboy,1474675200,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Three Stars Very good,2016-09-24,good
4,1,True,A212PQ0HQPNNWM,4126895493,Kelly Hales,1468713600,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Currently returning this product because the s...,2016-07-17,bad


In [ ]:
product_reviews_hp4[product_reviews_hp4.duplicated(['reviewerName','unixReviewTime','asin'],keep= False)]

,Rating,verified,reviewerID,asin,reviewerName,unixReviewTime,category,tech1,description,fit,...,feature,rank,also_view,main_cat,similar_item,date,price,review_text,time,rating_class
100,2,False,A3TM20GH659WDI,B00000J1FV,D. Price,1027900800,"['Electronics', 'Computers & Accessories', 'Co...",NaN,"['JENSEN JF-25, FULL-SIZE HEADPHONES WITH IN-L...",NaN,...,"['Full-range sound', '40-millimeter speaker dr...","['>#2,875,357 in Cell Phones & Accessories (Se...",[],Home Audio & Theater,NaN,"June 1, 1999",NaN,You get what you pay for... I was looking for ...,2002-07-29,bad
101,2,False,A3TM20GH659WDI,B00000J1FV,D. Price,1027900800,"['Electronics', 'Computers & Accessories', 'Co...",NaN,"['JENSEN JF-25, FULL-SIZE HEADPHONES WITH IN-L...",NaN,...,"['Full-range sound', '40-millimeter speaker dr...","['>#2,875,357 in Cell Phones & Accessories (Se...",[],Home Audio & Theater,NaN,"June 1, 1999",NaN,You get what you pay for... I was looking for ...,2002-07-29,bad
102,2,True,A2MKXP2VXKJUZU,B00000J1EJ,Bryan,1433548800,"['Electronics', 'Headphones']",NaN,['Microphones in the headphones detect ambient...,NaN,...,['Noise cancellation headphones help reduce am...,"['>#140,394 in Cell Phones & Accessories (See ...","['B00006JPRN', 'B075YR2PM4', 'B07J1LJS5K', 'B0...",Home Audio & Theater,"class=""a-bordered a-horizontal-stripes a-spa...","July 7, 2004",$2.00,This model has changed and not for the better ...,2015-06-06,bad
103,2,True,A2MKXP2VXKJUZU,B00000J1EJ,Bryan,1433548800,"['Electronics', 'Headphones']",NaN,['Microphones in the headphones detect ambient...,NaN,...,['Noise cancellation headphones help reduce am...,"['>#140,394 in Cell Phones & Accessories (See ...","['B00006JPRN', 'B075YR2PM4', 'B07J1LJS5K', 'B0...",Home Audio & Theater,"class=""a-bordered a-horizontal-stripes a-spa...","July 7, 2004",$2.00,This model has changed and not for the better ...,2015-06-06,bad
104,4,True,A2PPT56KT2JPB8,B00000J1EJ,K. Nelson,1431043200,"['Electronics', 'Headphones']",NaN,['Microphones in the headphones detect ambient...,NaN,...,['Noise cancellation headphones help reduce am...,"['>#140,394 in Cell Phones & Accessories (See ...","['B00006JPRN', 'B075YR2PM4', 'B07J1LJS5K', 'B0...",Home Audio & Theater,"class=""a-bordered a-horizontal-stripes a-spa...","July 7, 2004",$2.00,Four Stars happy with it,2015-05-08,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48945,2,False,A3THF5F2ZLWL6L,B00030CHRQ,Amidanshi,1113955200,"['Electronics', 'Headphones', 'Earbud Headphon...",NaN,['iPod friendly SONY MDR-EX71SLWK Headphones f...,NaN,...,['Extra-lightweight in-the-ear headphones for ...,"['>#1,058,088 in Cell Phones & Accessories (Se...","['B001EZYMF4', 'B00JRD13T8', 'B01JGTB1DU']",Home Audio & Theater,"class=""a-bordered a-horizontal-stripes a-spa...","September 4, 2004",NaN,Good sound quality; poorly made? I've bought t...,2005-04-20,bad
48946,2,True,A27V7PFFM2CAJH,B00030CHRQ,JPA,1112832000,"['Electronics', 'Headphones', 'Earbud Headphon...",NaN,['iPod friendly SONY MDR-EX71SLWK Headphones f...,NaN,...,['Extra-lightweight in-the-ear headphones for ...,"['>#1,058,088 in Cell Phones & Accessories (Se...","['B001EZYMF4', 'B00JRD13T8', 'B01JGTB1DU']",Home Audio & Theater,"class=""a-bordered a-horizontal-stripes a-spa...","September 4, 2004",NaN,"Great Sound, Terrible Design The sound from th...",2005-04-07,bad
48947,2,True,A27V7PFFM2CAJH,B00030CHRQ,JPA,1112832000,"['Electronics', 'Headphones', 'Earbud Headphon...",NaN,['iPod friendly SONY MDR-EX71SLWK Headphones f...,NaN,...,['Extra-lightweight in-the-ear headphones for ...,"['>#1,058,088 in Cell Phones & Accessories (Se...","['B001EZYMF4', 'B00JRD13T8', 'B01JGTB1DU']",Home Audio & Theater,"class=""a-bordered a-horizontal-stripes a-spa...","September 4, 2004",NaN,"Great Sound, Terrible Design The sound from th...",2005-04-07,bad
48948,2,False,A38U2M9OAEJAXJ,B00030CHRQ,The Groove,1111622400,"['Electronics', 'Headphones', 'Earbud Headphon...",NaN,['iPod friendly SONY MDR-EX71SLWK

In [ ]:
product_reviews_hp4 = product_reviews_hp4.drop_duplicates(['asin','reviewerName', 'unixReviewTime'], keep = 'first')
product_reviews_hp4

,Rating,verified,reviewerID,asin,reviewerName,unixReviewTime,category,tech1,description,fit,...,feature,rank,also_view,main_cat,similar_item,date,price,review_text,time,rating_class
0,5,True,A38RQFVQ1AKJQQ,4126895493,George Walker,1424563200,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Five Stars Great headphones. It's just the cor...,2015-02-22,good
1,5,True,A299MRB9O6GWDE,4126895493,Carolyn B,1494201600,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Officewear Really like these headphone. Wante...,2017-05-08,good
2,1,True,A3ACFC6DQQLIQT,4126895493,MK,1478304000,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,For the money they are fine. Just hope they ho...,2016-11-05,bad
3,3,True,A36BC0YFDBNB5X,4126895493,bigboy,1474675200,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Three Stars Very good,2016-09-24,good
4,1,True,A212PQ0HQPNNWM,4126895493,Kelly Hales,1468713600,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,['Brand new and High quality! Enjoy your favor...,NaN,...,['Adjustable headband allows you to create a c...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Currently returning this product because the s...,2016-07-17,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49638,5,True,AKXKYABFRI3KC,B00062PN6I,LGJ,1324252800,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,"['<div class=""aplus""> <h4>BEHRINGER HEADPHONES...",NaN,...,"['Ultra-wide frequency response', 'High dynami...","['>#1,988 in Musical Instruments (See Top 100 ...","['B000CCY3FO', 'B009HQHQ64', 'B0002Z82XA', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","April 13, 2004",NaN,fantastic Wow..... it has an amazing sound. Th...,2011-12-19,good
49639,5,False,A5MCDQ60DWUEV,B00062PN6I,Paul Lewis,1318809600,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,"['<div class=""aplus""> <h4>BEHRINGER HEADPHONES...",NaN,...,"['Ultra-wide frequency response', 'High dynami...","['>#1,988 in Musical Instruments (See Top 100 ...","['B000CCY3FO', 'B009HQHQ64', 'B0002Z82XA', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","April 13, 2004",NaN,Great value headphones. These are great headph...,2011-10-17,good
49640,5,True,A1LXE1JF4B4KQ4,B00062PN6I,fair reviews granted,1316131200,"['Electronics', 'Headphones', 'Over-Ear Headph...",NaN,"['<div class=""aplus""> <h4>BEHRINGER HEADPHONES...",NaN,...,"['Ultra-wide frequency response', 'High dynami...","['>#1,988 in Musical Instruments (See Top 100 ...","['B000CCY3FO', 'B009HQHQ64', 'B0002Z82XA', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","April 13, 2004"

In [ ]:
product_reviews_hp4=product_reviews_hp4.drop(['tech1','tech2','fit'],axis=1)

In [ ]:
product_reviews_hp4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25188 entries, 0 to 49642
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Rating          25188 non-null  int64         
 1   verified        25188 non-null  bool          
 2   reviewerID      25188 non-null  object        
 3   asin            25188 non-null  object        
 4   reviewerName    25187 non-null  object        
 5   unixReviewTime  25188 non-null  int64         
 6   category        25188 non-null  object        
 7   description     25188 non-null  object        
 8   title           25188 non-null  object        
 9   also_buy        25188 non-null  object        
 10  brand           25188 non-null  object        
 11  feature         25188 non-null  object        
 12  rank            25188 non-null  object        
 13  also_view       25188 non-null  object        
 14  main_cat        25188 non-null  object        
 15  si

In [ ]:
product_reviews_hp4.shape

(25188, 21)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import unicodedata
import contractions
from contractions import CONTRACTION_MAP
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\janya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\janya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\janya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from nltk.tokenize import word_tokenize
raw_tokens=len([w for t in (product_reviews_hp4["review_text"].apply(word_tokenize)) for w in t])
print('Number of raw tokens: {}'.format(raw_tokens))

Number of raw tokens: 2534634


In [ ]:
import re, string, unicodedata
import nltk
#import contractions
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from contractions import CONTRACTION_MAP

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text



# Define function to expand contractions
def expand_contractions(text):
    contractions_pattern = re.compile('({})'.format('|'.join(CONTRACTION_MAP.keys())),flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = CONTRACTION_MAP.get(match)\
                        if CONTRACTION_MAP.get(match)\
                        else CONTRACTION_MAP.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


# special_characters removal
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


def remove_punctuation_and_splchars(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_word = remove_special_characters(new_word, True)
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

stopword_list= stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
#stopword_list.remove('headphone')
#stopword_list.remove('headphones')
#stopword_list.remove('earbuds')
#stopword_list.remove('bud')
#stopword_list.remove('ear')
#stopword_list.remove('sony')
#stopword_list.remove('product')


def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopword_list:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation_and_splchars(words)
    words = remove_stopwords(words)
    return words

def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas


In [ ]:
def normalize_and_lemmaize(input):
    sample = denoise_text(input)
    sample = remove_special_characters(sample)
    words = nltk.word_tokenize(sample)
    words = normalize(words)
    lemmas = lemmatize(words)
    return ' '.join(lemmas)


In [ ]:
product_reviews_hp4['clean_text'] = product_reviews_hp4['review_text'].map(lambda text: normalize_and_lemmaize(text))

In [ ]:
product_reviews_hp4.head()

,Rating,verified,reviewerID,asin,reviewerName,unixReviewTime,category,description,title,also_buy,...,rank,also_view,main_cat,similar_item,date,price,review_text,time,rating_class,clean_text
0,5,True,A38RQFVQ1AKJQQ,4126895493,George Walker,1424563200,"['Electronics', 'Headphones', 'Over-Ear Headph...",['Brand new and High quality! Enjoy your favor...,HeadGear 3.5mm Foldable Headphone Headset for ...,"['B00W87LKXE', 'B019PO1ZRI', 'B019PO1ZMI', 'B0...",...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Five Stars Great headphones. It's just the cor...,2015-02-22,good,five star great headphones cord short
1,5,True,A299MRB9O6GWDE,4126895493,Carolyn B,1494201600,"['Electronics', 'Headphones', 'Over-Ear Headph...",['Brand new and High quality! Enjoy your favor...,HeadGear 3.5mm Foldable Headphone Headset for ...,"['B00W87LKXE', 'B019PO1ZRI', 'B019PO1ZMI', 'B0...",...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Officewear Really like these headphone. Wante...,2017-05-08,good,officewear really like headphone want somethin...
2,1,True,A3ACFC6DQQLIQT,4126895493,MK,1478304000,"['Electronics', 'Headphones', 'Over-Ear Headph...",['Brand new and High quality! Enjoy your favor...,HeadGear 3.5mm Foldable Headphone Headset for ...,"['B00W87LKXE', 'B019PO1ZRI', 'B019PO1ZMI', 'B0...",...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,For the money they are fine. Just hope they ho...,2016-11-05,bad,money fine hope hold wire headphone break less...
3,3,True,A36BC0YFDBNB5X,4126895493,bigboy,1474675200,"['Electronics', 'Headphones', 'Over-Ear Headph...",['Brand new and High quality! Enjoy your favor...,HeadGear 3.5mm Foldable Headphone Headset for ...,"['B00W87LKXE', 'B019PO1ZRI', 'B019PO1ZMI', 'B0...",...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Three Stars Very good,2016-09-24,good,three star good
4,1,True,A212PQ0HQPNNWM,4126895493,Kelly Hales,1468713600,"['Electronics', 'Headphones', 'Over-Ear Headph...",['Brand new and High quality! Enjoy your favor...,HeadGear 3.5mm Foldable Headphone Headset for ...,"['B00W87LKXE', 'B019PO1ZRI', 'B019PO1ZMI', 'B0...",...,"['>#316,475 in Cell Phones & Accessories (See ...","['B019PO1ZRI', 'B00W87LKXE', 'B019PO1ZNC', 'B0...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...",NaN,$7.99,Currently returning this product because the s...,2016-07-17,bad,currently return product sound not come curren...


In [ ]:
product_reviews_hp4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25188 entries, 0 to 49642
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Rating          25188 non-null  int64         
 1   verified        25188 non-null  bool          
 2   reviewerID      25188 non-null  object        
 3   asin            25188 non-null  object        
 4   reviewerName    25187 non-null  object        
 5   unixReviewTime  25188 non-null  int64         
 6   category        25188 non-null  object        
 7   description     25188 non-null  object        
 8   title           25188 non-null  object        
 9   also_buy        25188 non-null  object        
 10  brand           25188 non-null  object        
 11  feature         25188 non-null  object        
 12  rank            25188 non-null  object        
 13  also_view       25188 non-null  object        
 14  main_cat        25188 non-null  object        
 15  si

In [ ]:
# Let's put aside number of raw tokens in order to measure of cleaned tokens
from nltk.tokenize import word_tokenize
clean_tokens=len([w for t in (product_reviews_hp4["clean_text"].apply(word_tokenize)) for w in t])
#clean_tokens=len((product_reviews_hp4["clean_text"]))
print('Number of clean tokens: {}\n'.format(clean_tokens))
print('Percentage of removed tokens: {0:.2f}'.format(1-(clean_tokens/raw_tokens)))

Number of clean tokens: 1123901

Percentage of removed tokens: 0.56


In [ ]:
product_reviews_hp4.to_csv('\Datasets\Clean_Headphone_Dataset.csv', sep=',', encoding='utf-8', index = False)